# Pre-processing Montana Water Right data for WaDEQA upload.

Date Updated: 11/30/2020
Purpose:  To pre-process the Montana data into one master file for simple DataFrame creation and extraction.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Montana/WaterAllocation/RawInputData"
os.chdir(workingDir)

### Water Budget Data

In [2]:
#CSV input file
fileInput = "WaDE_PODs_input.csv"
df = pd.read_csv(fileInput)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997/11/25 11:45:00+00,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997/11/25 11:47:00+00,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960/12/31 00:00:00+00,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ


In [3]:
WaterSourceTypeCVDict = {
"SURFACE" : "Surface",
"GROUNDWATER" : "Groundwater",
"Unspecified" : "Unspecified",
"OTHER" : "Other",
"ALL NATURALLY OCCURING WATER": "All Naturally Occuring Water"
}

def retrieveWaterSourceTypeCV(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unspecified'
    else:
        String1 = colrowValue.strip()
        try:
            outList = WaterSourceTypeCVDict[String1]
        except:
            outList = 'Unspecified'
    return outList

df['inputWaterSourceTypeCV'] = df.apply(lambda row: retrieveWaterSourceTypeCV(row['SOURCE_TYPE']), axis=1)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997/11/25 11:45:00+00,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997/11/25 11:47:00+00,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960/12/31 00:00:00+00,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater


In [4]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
df['ENF_PRIORITY_DATE'] = pd.to_datetime(df['ENF_PRIORITY_DATE'])
df['ENF_PRIORITY_DATE'] = pd.to_datetime(df["ENF_PRIORITY_DATE"].dt.strftime('%m/%d/%Y'))
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater


In [5]:
# Creating easy MethodTypeCV retreival for AllocationsAmounts_fact sheet.

x = datetime.datetime(1973, 7, 1)
x

def createMethodTypeCV(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        if colrowValue < x:
            outList = "Adjudication"
        else:
            outList = "Appropriations"
    return outList

df['inputMethodTypeCV'] = df.apply(lambda row: createMethodTypeCV(row['ENF_PRIORITY_DATE']), axis=1)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV,inputMethodTypeCV
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface,Appropriations
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface,Appropriations
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater,Adjudication


In [6]:
# Creating TimeframeStart.
# Spliting string, returning WaDE friendly format.

MonthNumbDict = {
"Jan" : "01",
"Feb" : "02",
"Mar" : "03",
"May" : "04",
"Apr" : "05",
"Jun" : "06",
"Jul" : "07",
"Aug" : "08",
"Sep" : "09",
"Oct" : "10",
"Nov" : "11",
"Dec" : "12"}

def createTimeframeStart(ColRowVal):
    val = str(ColRowVal)
    day = val.split('-')[0]
    try:
        month = val.split('-')[1]
        month = MonthNumbDict[month] + "/"
    except:
        day = ""
        month = ""
    outlist = month + day
    return outlist

df['inputTimeframeStart'] = df.apply(lambda row: createTimeframeStart(row['PER_DIV_BGN_DT']), axis=1)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV,inputMethodTypeCV,inputTimeframeStart
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater,Adjudication,01/1


In [7]:
# Creating TimeframeEnd.
# Spliting string, returning WaDE friendly format.

MonthNumbDict = {
"Jan" : "01",
"Feb" : "02",
"Mar" : "03",
"May" : "04",
"Apr" : "05",
"Jun" : "06",
"Jul" : "07",
"Aug" : "08",
"Sep" : "09",
"Oct" : "10",
"Nov" : "11",
"Dec" : "12"}

def createTimeframeEnd(ColRowVal):
    val = str(ColRowVal)
    day = val.split('-')[0]
    try:
        month = val.split('-')[1]
        month = MonthNumbDict[month] + "/"
    except:
        day = ""
        month = ""
    outlist = month + day
    return outlist

df['inputTimeframeEnd'] = df.apply(lambda row: createTimeframeEnd(row['PER_DIV_END_DT']), axis=1)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV,inputMethodTypeCV,inputTimeframeStart,inputTimeframeEnd
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater,Adjudication,01/1,12/31


In [8]:
# Excempt Status
# Read in WR_Type, allow these water rights to contain NULL values.

def createExemptOfVolumeFlowPriority(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 0
    else:
        if colrowValue == "EXEMPT NOTICE":
            outList = 1
        else:
            outList = 0
    return outList

df['in_ExemptOfVolumeFlowPriority'] = df.apply(lambda row: createExemptOfVolumeFlowPriority(row['WR_TYPE']), axis=1)
df.head(3)

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV,inputMethodTypeCV,inputTimeframeStart,inputTimeframeEnd,in_ExemptOfVolumeFlowPriority
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.4,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1022850.406,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31,0
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.2,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1023781.369,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31,0
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1021011.474,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater,Adjudication,01/1,12/31,0


### Shapefile Data

In [9]:
# Shapefile input
ShapeFileInput = gpd.read_file('PODShp/MT_POD.shp')
dfshapetemp = pd.DataFrame(ShapeFileInput)
dfshapetemp.head(3)

,OBJECTID,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYP,SOURCE_NAM,HUC_12_SOU,HUC_12,MEANS_OF_D,ENF_PRIORI,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_AL,PER_DIV_AL,PER_DIV_BG,PER_DIV_EN,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_,LLDS_QTRS,LLDS_TRS,LLDS_COUNT,X_METERS,Y_METERS,Y_MAPPED,X_METERS_C,Y_METERS_C,LLDS_TWP_R,LLDS_SCTN_,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SE,VERSIDSEQ,PODV_ID_SE,DTM_CREATE,WR_PROGRAM,geometry
0,1,41H 150740 00,STATEMENT OF CLAIM,ACTIVE,DAVID H BOS; MARILYN H BOS,IRRIGATION,1,SURFACE,HYALITE CREEK,Lower Hyalite Creek,100200081002,HEADGATE,1868-06-30,B-F-K DITCH,561.00,1.25,0.0,300.0,04/01 to 10/19,04/01 to 10/19,04/01,10/19,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,POST DECREE,308779-2,0,NWNWNW,3S5E 14,GALLATIN,476460.46759,149467.83631,Y,0.0000,0.0000,3S5E,14,2014682,234311,2016-02-04,41H,150740,00,308779,2,723165,2019-10-08,ADJ,POINT (-111.08358 45.58365)
1,2,76H 47452 00,STATEMENT OF CLAIM,ACTIVE,GEORGE H HERSHBERGER; MARY E HERSHBERGER,IRRIGATION,1,SURFACE,BIG CREEK,Big Creek,170102051201,HEADGATE,1882-04-01,SANDERS DITCH B,45.22,0.10,0.0,10.0,06/01 to 10/31,06/01 to 10/31,06/01,10/31,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,91885-1,0,SESESE,8N21W 12,RAVALLI,0.00000,0.00000,N,242620.4347,256055.2440,8N21W,12,1790623,221105,1973-07-01,76H,47452,00,91885,1,502304,2019-10-08,ADJ,POINT (-114.15749 46.45860)
2,3,76H 105159 00,STATEMENT OF CLAIM,ACTIVE,POOR BROTHERS LLC; BRANDON TINTZMAN; MEGAN TIN...,IRRIGATION,1,SURFACE,SLEEPING CHILD CREEK,Lower Sleeping Child Creek,170102050705,HEADGATE,1880-06-19,MASON-HARMONY DITCH,150.00,0.33,0.0,16.0,05/01 to 10/15,05/01 to 10/15,05/01,10/15,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,185488-1,3,NWNWSW,5N20W 30,RAVALLI,0.00000,0.00000,N,241031.9705,222795.2968,5N20W,30,1999364,260120,1973-07-01,76H,105159,00,185488,1,505127,2019-10-08,ADJ,POINT (-114.15242 46.15890)


In [10]:
columnsList = ['WR_NUMBER', 'POD_NO', 'DITCH_NAME', 'geometry']
dfshape = pd.DataFrame(columns=columnsList)
dfshape['WR_NUMBER'] = dfshapetemp['WR_NUMBER']
dfshape['POD_NO'] = dfshapetemp['POD_NO']
dfshape['DITCH_NAME'] = dfshapetemp['DITCH_NAME']
dfshape['geometry'] = dfshapetemp['geometry']
dfshape = dfshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfshape.head(3)

,WR_NUMBER,POD_NO,DITCH_NAME,geometry
0,41H 150740 00,1,B-F-K DITCH,POINT (-111.08358 45.58365)
1,76H 47452 00,1,SANDERS DITCH B,POINT (-114.15749 46.45860)
2,76H 105159 00,1,MASON-HARMONY DITCH,POINT (-114.15242 46.15890)


### Inspect Output Data & Export

In [11]:
df.dtypes

X                                       float64
Y                                       float64
WR_NUMBER                                object
WR_TYPE                                  object
WR_STATUS                                object
ALL_OWNERS                               object
PURPOSES                                 object
POD_NO                                    int64
SOURCE_TYPE                              object
SOURCE_NAME                              object
HUC_12_SOURCE                            object
HUC_12                                  float64
MEANS_OF_DIV                             object
ENF_PRIORITY_DATE                datetime64[ns]
DITCH_NAME                               object
FLW_RT_GPM                              float64
FLW_RT_CFS                              float64
VOLUME                                  float64
MAX_ACRES                               float64
PER_USE_ALL                              object
PER_DIV_ALL                             

In [12]:
df

,X,Y,WR_NUMBER,WR_TYPE,WR_STATUS,ALL_OWNERS,PURPOSES,POD_NO,SOURCE_TYPE,SOURCE_NAME,HUC_12_SOURCE,HUC_12,MEANS_OF_DIV,ENF_PRIORITY_DATE,DITCH_NAME,FLW_RT_GPM,FLW_RT_CFS,VOLUME,MAX_ACRES,PER_USE_ALL,PER_DIV_ALL,PER_DIV_BGN_DT,PER_DIV_END_DT,ABST_LINK,VERS_TYPE,WRKEY,LLDS_GOVT_LOT,LLDS_QTRS,LLDS_TRS,LLDS_COUNTY_NAME,X_METERS,Y_METERS,Y_MAPPED,X_METERS_CACULATED,Y_METERS_CACULATED,LLDS_TWP_RNG,LLDS_SCTN_NO,LLDSIDSEQ,TRSSIDSEQ,OP_AUTH_DT,BASIN,WR_NO,EXT,WRGT_ID_SEQ,VERSIDSEQ,PODV_ID_SEQ,DTM_CREATED,WR_PROGRAM,inputWaterSourceTypeCV,inputMethodTypeCV,inputTimeframeStart,inputTimeframeEnd,in_ExemptOfVolumeFlowPriority
0,-104.122561,45.106454,38H 102244 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.40,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180441-1,NaN,SWNESE,8S62E 28,CARTER,NaN,NaN,N,1.022850e+06,109714.7502,8S62E,28,1048761,245390,12/5/1997,38H,102244,0.0,180441,1,249593,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31,0
1,-104.112287,45.090173,38H 102245 00,STOCKWATER PERMIT,ACTIVE,USA (DEPT OF INTERIOR BUREAU OF LAND MGMT),STOCK,1,SURFACE,UNNAMED TRIBUTARY OF SOUTH INDIAN CREEK,Upper South Indian Creek,1.010000e+11,PIT/DAM,1997-11-25,NaN,NaN,NaN,5.20,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,180442-1,NaN,NWSESW,8S62E 34,CARTER,NaN,NaN,N,1.023781e+06,107965.3701,8S62E,34,1048762,245440,12/5/1997,38H,102245,0.0,180442,1,249594,2019/10/08 16:22:49+00,NaN,Surface,Appropriations,01/1,12/31,0
2,-104.150532,45.059367,38H 126342 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,GROUNDWATER,GROUNDWATER,Upper South Indian Creek,1.010000e+11,WELL,1960-12-31,NaN,6.0,0.01,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214400-1,NaN,SESESW,9S62E 8,CARTER,NaN,NaN,N,1.021011e+06,104343.6560,9S62E,8,1456985,245554,7/1/1973,38H,126342,0.0,214400,1,229310,2019/10/08 16:22:49+00,ADJ,Groundwater,Adjudication,01/1,12/31,0
3,-104.150546,45.064799,38H 126344 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,SURFACE,UNNAMED TRIBUTARY OF WATSON DRAW,Upper South Indian Creek,1.010000e+11,DAM,1933-12-31,NaN,NaN,NaN,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,214402-1,NaN,NENESW,9S62E 8,CARTER,NaN,NaN,N,1.020969e+06,104945.8748,9S62E,8,1858032,245554,7/1/1973,38H,126344,0.0,214402,1,249756,2019/10/08 16:22:49+00,ADJ,Surface,Adjudication,01/1,12/31,0
4,-104.135229,45.063903,38H 126345 00,STATEMENT OF CLAIM,ACTIVE,"JAMES R SCHMIDT, MARIAN J SCHMIDT",STOCK,1,SURFACE,UNNAMED TRIBUTARY OF WATSON DRAW,Upper South Indian Creek,1.010000e+11,DAM,1963-12-31,NaN,NaN,NaN,NaN,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,305588-1,NaN,E2NWSW,9S62E 9,CARTER,NaN,NaN,N,1.022179e+06,104928.9778,9S62E,9,1858033,189017,7/1/1973,38H,126345,0.0,305588,1,249757,2019/10/08 16:22:49+00,ADJ,Surface,Adjudication,01/1,12/31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473795,-115.587065,47.852258,76N 99889 00,GROUND WATER CERTIFICATE,ACTIVE,HAROLD W HUDSON,DOMESTIC,1,GROUNDWATER,GROUNDWATER,Noxon Reservoir-Belgian Gulch,1.700000e+11,WELL,1996-12-27,NaN,35.0,0.07,1.63,NaN,01/01 to 12/31,01/01 to 12/31,1-Jan,31-Dec,http://wr.dnrc.mt.gov/reports/rwservlet?dnrcwr...,ORIGINAL RIGHT,176364-1,NaN,NWSW,24N31W 9,SANDERS,NaN,NaN,N,1.451420e+05,417959.1993,24N31W,9,1040381,148131,5/5/1997,76N,99889,0.0,176364,1,212736,2019/10/08 16:22:49+00,NaN,Groundwater,Appropriations,01/1,12

In [13]:
dfshape.dtypes

WR_NUMBER       object
POD_NO           int64
DITCH_NAME      object
geometry      geometry
dtype: object

In [14]:
dfshape

,WR_NUMBER,POD_NO,DITCH_NAME,geometry
0,41H 150740 00,1,B-F-K DITCH,POINT (-111.08358 45.58365)
1,76H 47452 00,1,SANDERS DITCH B,POINT (-114.15749 46.45860)
2,76H 105159 00,1,MASON-HARMONY DITCH,POINT (-114.15242 46.15890)
3,76H 103763 00,1,SANDERS DITCH B,POINT (-114.15749 46.45860)
4,76H 116689 00,1,SANDERS DITCH B,POINT (-114.15749 46.45860)
...,...,...,...,...
547979,41QJ 97432 00,1,GANS KLEIN DITCH,POINT (-112.30455 46.81050)
547980,41QJ 128020 00,1,GANS KLEIN DITCH,POINT (-112.30455 46.81050)
547981,43D 10239 00,1,HAARA DITCH,POINT (-109.28904 45.16001)
547982,41QJ 5581 00,1,GANS KLEIN DITCH,POINT (-112.30455 46.81050)


In [15]:
# Export out to CSV.
df.to_csv('P_MontanaMaster.csv', index=False) # The output.
dfshape.to_csv('P_MontanaGeometry.csv', index=False) # The output geometry.